## Connect Four
This is a recreation of the classic [connect 4](https://en.wikipedia.org/wiki/Connect_Four) game by Milton Bradley.  

**Rules for Beginners:**
- [Overview of gameplay](https://en.wikipedia.org/wiki/Connect_Four#Gameplay)  
- [Demo](https://en.wikipedia.org/wiki/Connect_Four#/media/File:Connect_Four.gif)

In [1]:
import numpy as np
import itertools

In [2]:
def place_piece(board, color, position):
    '''place piece in given column and fall to lowest empty spot'''
    i = 1
    while True:
        #must take position-1 so that player can index 1-7
        if not board[-i,position-1]:
            placement = [6-i, position-1] #6 rows, place at lowest available
            board[-i,position-1] = color
            break
        i += 1
    return board, placement

In [3]:
def find_diag(row, col, board):
    '''identify diagonal lines'''
    offset = min(row,col)
    row = row-offset
    col = col-offset

    diag = []
    while row<6 and col<7:
        diag.append(board[row,col])
        row += 1
        col += 1
    return diag

def find_diags(board,placement):
    '''points in direction of each diagonal, calls find_diag for each dir'''
    row = placement[0]
    col = placement[1]
    diag1 = find_diag(row,col,board)
    #instead of having separate find_diag functions, we can reuse the function
    #to do so, we must simply invert the rows and the row index
    row = 5-row 
    diag2 = find_diag(row,col,board[::-1,:]) 
    return diag1, diag2

In [4]:
def check_line(lines, color):
    '''evaluates each direction to see if player won'''
    for line in lines:
        #analyze consecutive occurences in line
        #ex: ['_' '_' 'R' 'R' 'B' '_' 'R'] returns [2,1,1]
        wins = [sum(1 for _ in group) for key, group in itertools.groupby(line) if key]
        if max(wins)>3:
            return True
    return False

def check_winner(board, placement, color):
    '''establishes the lines in which the last placement could have won'''
    row = board[placement[0],:]
    col = board[:,placement[1]]
    
    diag1, diag2 = find_diags(board, placement)
    
    lines = [row, col, diag1, diag2] 
    return check_line(lines, color)

In [5]:
def get_ansi(color):
    '''ANSI code for Red or Blue'''
    if color == 'Red':
        return 31
    return 34

In [6]:
def turn(board, color):
    '''calls functions to perform entire turn'''
    display_board(board)
    
    while True:
        ansi = get_ansi(color)
        position = input(f"\x1b[{ansi}m{color} player please enter your choice for where you will place your piece (1-7):")
        try:
            position = int(position)
        except: #if player input is invalid
            position = -1
        if (position < 1) or (position > 7):
            print("Your input was invalid. Please input a value from 1-7.")
        elif board[0,position-1]:
            print("You chose a full column. Please choose a different column.")
        else:
            break
        
    board, placement = place_piece(board, color, position)
    winner = check_winner(board, placement, color)
    return winner

In [7]:
def display_board(board):
    pretty_board = board.copy()
    print(np.where(pretty_board=='', '_', pretty_board)) 
    cols = np.array([[1,2,3,4,5,6,7]]).astype(str)
    print(cols,'\n')

In [8]:
board = np.zeros((6,7), dtype='str')
winner = False
color = 'Blue'

print("Let's play Connect Four!\n")
print("Players are red (R) and blue (B) - red goes first.\n")

while winner is False:
    if color == 'Blue':color = 'Red'
    else: color = 'Blue'
    winner = turn(board, color)
    if board[board==''].size==0:
        break

print(f'\n***Final board***')
display_board(board)
ansi = get_ansi(color)
if winner is True:
    print(f'\n\x1b[{ansi}m{color} player has won!')
else:
    print('\nGame ends in a tie - there are no spaces left.')

Let's play Connect Four!

Players are red (R) and blue (B) - red goes first.

[['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']]
[['1' '2' '3' '4' '5' '6' '7']] 

Red player please enter your choice for where you will place your piece (1-7):1
[['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['R' '_' '_' '_' '_' '_' '_']]
[['1' '2' '3' '4' '5' '6' '7']] 

Blue player please enter your choice for where you will place your piece (1-7):4
[['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['R' '_' '_' 'B' '_' '_' '_']]
[['1' '2' '3' '4' '5' '6' '7']] 

Red player please enter your choice for where you will place your piece (1-7):3
[['_' '_' '_' '_' 